In [2]:
import tensorflow as tf

In [1]:
import os


os.getcwd()

'f:\\Machine_learning-End to end projects\\CNN_Deep_learning_project\\research'

In [3]:
import os
os.chdir("../")

In [18]:
## For loo

os.environ["MLFLOW_TRACKING_URI"]="https://github.com/deveshpatil619/CNN_Deep_learning_project"
os.environ["MLFLOW_TRACKING_USERNAME"]="deveshpatil619"
os.environ["MLFLOW_TRACKING_PASSWORD"]="84215e85b5e87347572d9d272c798b2b1ff2a546"

In [19]:
model = tf.keras.models.load_model("artifacts/training/model.h5")  ## our model path

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) #"frozen" means an object cannot be modified
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    

In [22]:
from DeepClassifier.constants import *
from DeepClassifier.utils import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:  ##it will return EvaluationConfig
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PetImages",
            mlflow_uri="https://dagshub.com/deveshpatil619/CNN_Deep_learning_project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [23]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse ##The URL parsing functions focus on splitting a URL string into its components, or on combining URL components into a URL string.

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):  ## here we have kept this method hidden

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:  ## we will simply load the model and return type of model will be tf.keras.Model
        return tf.keras.models.load_model(path) ## we will return the loaded model


    def evaluation(self):  ## for model evaluation
        self.model = self.load_model(self.config.path_of_model) ## model path
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator) ## passing the valid_generator to the evaluation

    def save_score(self):  ## we will save the loss value as well as the accuracy
        scores = {"loss": self.score[0], "accuracy": self.score[1]} ## these scores are metrics
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri) ## setting the mlflow uri
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme ##Set the tracking server URI
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [24]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)  ## we will use the get_validation_config in Evaluation class
    evaluation.evaluation()  ## we will call the evaluation method
    evaluation.save_score()  ## we will call the save score method
    evaluation.log_into_mlflow()  ## we will call the log_into_mlflow method
    
except Exception as e:
   raise e

## it will log all the metrics as well as the parameters

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 2888s 6s/step - loss: 2.0474 - accuracy: 0.8397


MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 422 != 200. Response body: '<!DOCTYPE html>
<html>
  <head>
    <meta http-equiv="Content-type" content="text/html; charset=utf-8">
    <meta http-equiv="Content-Security-Policy" content="default-src 'none'; base-uri 'self'; connect-src 'self'; form-action 'self'; img-src 'self' data:; script-src 'self'; style-src 'unsafe-inline'">
    <meta content="origin" name="referrer">
    <title>Oh no &middot; GitHub</title>
    <meta name="viewport" content="width=device-width">
    <style type="text/css" media="screen">
      body {
        background-color: #f6f8fa;
        color: #24292e;
        font-family: -apple-system,BlinkMacSystemFont,Segoe UI,Helvetica,Arial,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol;
        font-size: 14px;
        line-height: 1.5;
        margin: 0;
      }

      .container { margin: 50px auto; max-width: 600px; text-align: center; padding: 0 24px; }

   